In [5]:
%pip install requests
%pip install pandas
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -------- ------------------------------- 30.7/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\SSAFY\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 🎈 테스트

### **상단 10건을 조회하여 한 파일에 저장**

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

joongang_news_df = pd.DataFrame(columns=['headline', 'time', 'content', 'reporter'])
idx = 0
header = {'User-Agent' : ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                          '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

# 스크랩 함수
def scrape_article(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        category = soup.find('a', { 'class': 'title' }).get_text(strip=True)
        if category != '정치': # 정치가 아니면 Skip
            print(f"Skipping article {article_url}: not a Politics article.")
            return None
    except AttributeError:
        print(f"Skipping article {article_url}: category not found.")
        return None

    try:
        headline = soup.find('h1', { 'class': 'headline' }).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    try:
        date = soup.find('p', { 'class': 'date' })
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    try:
        content = soup.find('div', { 'class': 'article_body' }).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    try:
        byline = soup.find('div', { 'class': 'byline'})
        reporter = byline.find('a').get_text(strip=True).split('\n')[0]
    except AttributeError:
        reporter = 'N/A'

    return headline, time, content, reporter

# 스크랩 이후 중복 데이터 제거 후처리 함수
def remove_duplicates_by_time(df):
    # Remove rows where the 'time' field is the same, keeping the first occurrence
    df = df.drop_duplicates(subset='time', keep='first').reset_index(drop=True)
    return df

# Base URL
base_url = "https://www.joongang.co.kr/politics"
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

cards = soup.find_all('li', class_='card')
for card in cards:
    article_link = card.find('a', href=True)['href']
    print(f"Scraping article: {article_link}")

    # 1차 결과 검증
    result = scrape_article(article_link)

    if result is None:
        continue

    headline, time, content, reporter = result

    # 2차 결과 검증
    if 'N/A' in [headline, time, content, reporter]:
        print(f"Skipping article because {[headline, time, content, reporter]}: missing required fields.")
        continue

    joongang_news_df.loc[idx] = [headline, time, content, reporter]
    idx += 1

# 최신순 정렬
# 1. time의 데이터를 datetime 형식으로 변경
joongang_news_df['time'] = pd.to_datetime(joongang_news_df['time'])

# 2. 시간 기준 내림차순 정렬
joongang_news_df = joongang_news_df.sort_values(by='time', ascending=False)

# 3. 시간 기준으로 중복 제거
joongang_news_df = remove_duplicates_by_time(joongang_news_df)

# 4. 다시 time 형식 변경
joongang_news_df['time'] = joongang_news_df['time'].dt.strftime('%Y-%m-%dT%H:%M:%S%z')

joongang_news_df.to_json('./joongang-articles/joongang_articles.json', orient='records', lines=True, force_ascii=False)

Scraping article: https://www.joongang.co.kr/article/25278499
Skipping article https://www.joongang.co.kr/article/25278499: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278695
Skipping article https://www.joongang.co.kr/article/25278695: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278716
Skipping article https://www.joongang.co.kr/article/25278716: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278507
Skipping article https://www.joongang.co.kr/article/25278507: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278705
Scraping article: https://www.joongang.co.kr/artic

## 🎈 테스트

### 1건씩 조회하여 단일 기사별 파일 생성
---

1. 파일 type: json

2. 파일명 형식: uuid

3. 필드

    - headline
    
    - time -> '%Y-%m-%dT%H:%M:%S%z'
    
    - content

    - reporter

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import os

# 예외 처리
output_dir = './joongang-articles'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

header = {'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'),}

# 스크랩 함수
def scrape_article(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        if category != '정치':  # 정치가 아니면 패스
            print(f"Skipping article {article_url}: not a Politics article.")
            return None
    except AttributeError:
        print(f"Skipping article {article_url}: category not found.")
        return None

    try:
        headline = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    try:
        byline = soup.find('div', {'class': 'byline'})
        reporter = byline.find('a').get_text(strip=True).split('\n')[0]
    except AttributeError:
        reporter = 'N/A'

    return {'headline': headline, 'time': time, 'content': content, 'reporter': reporter}

# 파일 생성
def save_article_as_json(article_data):
    file_name = str(uuid.uuid4()) + '.json'
    file_path = os.path.join(output_dir, file_name)
    
    # 단일 dataframe 생성
    single_article_df = pd.DataFrame([article_data])
    
    # json으로 변환
    single_article_df.to_json(file_path, orient='records', lines=True, force_ascii=False)
    print(f"Article saved to {file_path}")

# 새로운 기사 스크랩
def check_for_new_articles():
    base_url = "https://www.joongang.co.kr/politics"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    cards = soup.find_all('li', class_='card')
    for card in cards:
        article_link = card.find('a', href=True)['href']
        print(f"Scraping article: {article_link}")

        # Scrape the article
        result = scrape_article(article_link)

        if result is None:
            continue

        # Save each article individually
        save_article_as_json(result)

# 실행
while True:
    check_for_new_articles()
    print("Waiting for new articles...")
    time.sleep(600)  # 10분에 한 번

Scraping article: https://www.joongang.co.kr/article/25278499
Skipping article https://www.joongang.co.kr/article/25278499: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278695
Skipping article https://www.joongang.co.kr/article/25278695: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278507
Skipping article https://www.joongang.co.kr/article/25278507: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278788
Skipping article https://www.joongang.co.kr/article/25278788: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278704
Skipping article https://www.joongang.co.kr/article/25278704: not a Politics article.
Scraping article: https://www.joongang.co.kr/article/25278705
Article saved to ./joongang-articles\7256080b-dda7

## 🎈 테스트

### S3 업로드

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import uuid
import boto3
import os

# S3 setup (ensure AWS credentials are configured via AWS CLI or environment variables)
s3 = boto3.client('s3')
bucket_name = 'your-s3-bucket-name'

# Scrape article function
def scrape_article(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        category = soup.find('a', {'class': 'title'}).get_text(strip=True)
        if category != '정치':  # Skip non-Politics articles
            print(f"Skipping article {article_url}: not a Politics article.")
            return None
    except AttributeError:
        print(f"Skipping article {article_url}: category not found.")
        return None

    try:
        headline = soup.find('h1', {'class': 'headline'}).get_text(strip=True)
    except AttributeError:
        headline = 'N/A'

    try:
        date = soup.find('p', {'class': 'date'})
        time = date.find('time').get('datetime')
    except AttributeError:
        time = 'N/A'

    try:
        content = soup.find('div', {'class': 'article_body'}).get_text(strip=True)
    except AttributeError:
        content = 'N/A'

    try:
        byline = soup.find('div', {'class': 'byline'})
        reporter = byline.find('a').get_text(strip=True).split('\n')[0]
    except AttributeError:
        reporter = 'N/A'

    return {'headline': headline, 'time': time, 'content': content, 'reporter': reporter}

# Save article as JSON and upload to S3
def save_article_to_s3(article_data):
    # Generate UUID for the file name
    file_name = str(uuid.uuid4()) + '.json'
    json_data = pd.DataFrame([article_data])

    # Save locally before uploading to S3
    local_file_path = '/tmp/' + file_name  # Using /tmp to avoid filling local storage
    json_data.to_json(local_file_path, orient='records', lines=True, force_ascii=False)

    # Upload the JSON file to S3
    try:
        s3.upload_file(local_file_path, bucket_name, f'joongang-articles/{file_name}')
        print(f"Uploaded {file_name} to S3 bucket {bucket_name}")
    except Exception as e:
        print(f"Error uploading {file_name}: {e}")

    # Optionally, remove the local file after upload
    os.remove(local_file_path)

# Check for new articles
def check_for_new_articles():
    base_url = "https://www.joongang.co.kr/politics"
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    cards = soup.find_all('li', class_='card')
    for card in cards:
        article_link = card.find('a', href=True)['href']
        print(f"Scraping article: {article_link}")

        # Scrape the article
        result = scrape_article(article_link)

        if result is None:
            continue

        # Save and upload article to S3
        save_article_to_s3(result)

# Continuously monitor the site for new articles
while True:
    check_for_new_articles()
    print("Waiting for new articles...")
    time.sleep(600)  # Wait for 10 minutes before checking again
